# Impots

In [139]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from prophet import Prophet
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
import joblib


# 1. Carregar base de dados via URL

In [ ]:

# Definir a URL
url = "http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view"

# Fazer a requisição à página
response = requests.get(url)

if response.status_code == 200:
    print(" Página acessada com sucesso.")
else:
    try:
        df = pd.read_csv("dados/dados_petroleo_brent_2005_2025.csv")
        print("Arquivo carregado com sucesso!")
    except FileNotFoundError:
        print(f"Erro: O arquivo local {df} não foi encontrado.")
        df = None  
    print(f" Erro ao acessar a página. Código: {response.status_code}")

# 2. Parsear o HTML e localizar a tabela

In [ ]:

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    # Parsear o HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Procurar a tabela com a classe específica
    table = soup.find('table', {'class': 'dxgvTable'})
    
    if table:
        print(" Tabela encontrada, iniciando a criação do DataFrame...")
    else:
        print(" Tabela não encontrada na página.")

# 3. Criar o DataFrame inicial

In [ ]:
if table:
    # Ler a tabela HTML para um DataFrame Pandas
    df = pd.read_html(str(table))[0]

    # Renomear as colunas
    df.columns = ["Data", "Preço (US$)"]

    # Exibir as 5 primeiras linhas para conferir
    df.head()

# 4. Limpeza inicial dos dados

In [ ]:
# **4. Limpeza inicial dos dados**
# Filtrar apenas linhas onde a coluna 'Data' é válida
print("Limpando dados inválidos na coluna 'Data'...")
df = df[df["Data"].str.match(r"\d{2}/\d{2}/\d{4}", na=False)]

# Converter a coluna "Data" para datetime
df["Data"] = pd.to_datetime(df["Data"], format="%d/%m/%Y", errors="coerce")

# Remover valores nulos gerados pela conversão
df = df.dropna(subset=["Data"]).reset_index(drop=True)

# Exibir informações sobre o DataFrame
df.info()

 # 5. Ajuste dos valores na coluna “Preço (US$)”

In [ ]:
# Substituir a vírgula por ponto e converter para float
df["Preço (US$)"] = df["Preço (US$)"].str.replace(",", ".").astype(float) / 100

# Exibir estatísticas descritivas
df.describe()

#  6. Filtrar os dados no intervalo de 2005 a 2025

In [ ]:
# Manter apenas os dados entre 2005 e 2025
df = df[(df["Data"] >= "2005-01-01") & (df["Data"] <= "2025-12-31")]

# Resetar o índice
df = df.reset_index(drop=True)

# Exibir as 5 primeiras linhas após a filtragem
df.head()

 # 7. Verificação da Qualidade dos Dados

In [ ]:
# Exibir estatísticas descritivas
print(" Estatísticas descritivas:")
display(df.describe())

# Verificar a contagem de valores nulos
print("\n Contagem de valores nulos:")
print(df.isnull().sum())

# Mostrar informações do DataFrame
print("\n Estrutura do DataFrame:")
df.info()

# Verificar se há datas duplicadas
duplicadas = df["Data"].value_counts()
if any(duplicadas > 1):
    print("\n TEM DATA DUPLICADAS!")
else:
    print("\n NÃO TEM DATA DUPLICADAS!")

# 8. Salvar os dados tratados

In [ ]:
# Salvar os dados limpos em CSV
df.to_csv("dados/dados_petroleo_brent_2005_2025.csv", index=False, encoding="utf-8")
print(f"Dados salvos com sucesso em {df}.")

# 10. Evolução do Preço ao Longo do Tempo

In [ ]:
# Evolução do preço ao longo do tempo
plt.figure(figsize=(12, 6))
sns.lineplot(x=df["Data"], y=df["Preço (US$)"], linewidth=2)
plt.title("Evolução do Preço do Petróleo Brent (2005-2025)")
plt.xlabel("Ano")
plt.ylabel("Preço (US$)")
plt.grid(True)
plt.show()

 # 11. Distribuição dos Preços

In [ ]:

plt.figure(figsize=(12, 6))
sns.histplot(df["Preço (US$)"], bins=30, kde=True)
plt.title("Distribuição dos Preços do Petróleo Brent")
plt.xlabel("Preço (US$)")
plt.ylabel("Frequência")
plt.grid(True)
plt.show()

# 12. Boxplot dos Preços ao Longo do Tempo

In [ ]:
# 📦 Boxplot dos preços ao longo do tempo
plt.figure(figsize=(12, 6))
sns.boxplot(x=df["Data"].dt.year, y="Preço (US$)", data=df)
plt.title("Boxplot dos Preços do Petróleo por Ano")
plt.xlabel("Ano")
plt.ylabel("Preço (US$)")
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

# 13. Preço Médio por Ano

In [ ]:

df_yearly = df.groupby(df["Data"].dt.year)["Preço (US$)"].mean().reset_index()
plt.figure(figsize=(12, 6))
sns.barplot(x="Data", y="Preço (US$)", data=df_yearly)
plt.title("Preço Médio Anual do Petróleo Brent")
plt.xlabel("Ano")
plt.ylabel("Preço Médio (US$)")
plt.grid(True)
plt.show()

# 14. Evolução do Preço com Eventos Geopolíticos

In [ ]:

plt.figure(figsize=(12, 6))
sns.lineplot(x="Data", y="Preço (US$)", data=df, label="Preço do Petróleo")
plt.axvline(pd.Timestamp("2008-09-15"), color="red", linestyle="--", label="Crise Financeira 2008")
plt.axvline(pd.Timestamp("2020-03-01"), color="blue", linestyle="--", label="Início da Pandemia COVID-19")
plt.title("Evolução do Preço do Petróleo Brent com Eventos Geopolíticos")
plt.xlabel("Ano")
plt.ylabel("Preço (US$)")
plt.legend()
plt.grid(True)
plt.show()

 # 15. Variação Percentual Mensal do Preço

In [ ]:

df["Variação (%)"] = df["Preço (US$)"].pct_change() * 100
plt.figure(figsize=(12, 6))
sns.lineplot(x="Data", y="Variação (%)", data=df)
plt.axhline(0, color="black", linestyle="--")
plt.title("Variação Percentual Mensal do Preço do Petróleo")
plt.xlabel("Ano")
plt.ylabel("Variação (%)")
plt.grid(True)
plt.show()

# 16. Comparação Antes e Depois da Crise de 2008

In [ ]:

df_crise_2008 = df[(df["Data"] >= "2007-01-01") & (df["Data"] <= "2009-12-31")]
plt.figure(figsize=(12, 6))
sns.lineplot(x="Data", y="Preço (US$)", data=df_crise_2008)
plt.axvline(pd.Timestamp("2008-09-15"), color="red", linestyle="--", label="Crise Financeira 2008")
plt.title("Preço do Petróleo Antes e Após a Crise de 2008")
plt.xlabel("Ano")
plt.ylabel("Preço (US$)")
plt.legend()
plt.grid(True)
plt.show()

17. Decomposição da Série Temporal do Preço do Petróleo

In [ ]:

df = pd.read_csv("dados/dados_petroleo_brent_2005_2025.csv")
df["Data"] = pd.to_datetime(df["Data"], format="%Y-%m-%d", errors="coerce")


df.set_index("Data", inplace=True)


decomposition = seasonal_decompose(df["Preço (US$)"], model="additive", period=365)


plt.figure(figsize=(12, 10))


plt.subplot(4, 1, 1)
plt.plot(df["Preço (US$)"], label="Série Original", color="orange")
plt.legend()
plt.title("Decomposição da Série Temporal - Preço do Petróleo Brent")


plt.subplot(4, 1, 2)
plt.plot(decomposition.trend, label="Tendência", color="red")
plt.legend()


plt.subplot(4, 1, 3)
plt.plot(decomposition.seasonal, label="Sazonalidade", color="green")
plt.legend()


plt.subplot(4, 1, 4)
plt.plot(decomposition.resid, label="Resíduo", color="purple")
plt.legend()


plt.tight_layout()
plt.show()

# 18. Testando o Modelo Prophet para Previsão do Preço do Petróleo Brent

In [ ]:
# Carregar os dados históricos
try:
    df = pd.read_csv("dados/dados_petroleo_brent_2005_2025.csv")
    print("Arquivo carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    raise

# Converter a coluna "Data" para o formato datetime
df["Data"] = pd.to_datetime(df["Data"])

# Ordenar os dados por data
df = df.sort_values(by="Data")

# Exibir as 5 primeiras linhas para conferência
df.head()


#3. Divisão dos Dados em Treino e Teste

# Definir período de treino e teste
train = df[df["Data"] < "2023-01-01"].copy()
test = df[df["Data"] >= "2023-01-01"].copy()

# Exibir quantidades de dados
print(f"Dados de treino: {len(train)} linhas")
print(f"Dados de teste: {len(test)} linhas")


# Renomear colunas para o formato do Prophet
df_prophet = df.rename(columns={"Data": "ds", "Preço (US$)": "y"})

# Criar e treinar o modelo Prophet
model = Prophet()
model.fit(df_prophet)


# Converter conjunto de teste para o formato do Prophet
future_test = test.rename(columns={"Data": "ds"})

# Gerar previsões apenas para o período de teste
forecast_test = model.predict(future_test)

# Adicionar as previsões ao DataFrame de teste
test["Previsão"] = forecast_test["yhat"].values

# Exibir as 5 primeiras previsões
test.head()

In [ ]:
# Gerar previsões para o futuro (1 ano)
future = model.make_future_dataframe(periods=360)
forecast = model.predict(future)

# Gráfico das previsões do modelo Prophet
fig, ax = plt.subplots(figsize=(12, 6))
model.plot(forecast, ax=ax)
ax.legend(["Histórico", "Previsão", "Intervalo de Confiança"])
plt.title("Modelo Prophet - Previsão do Preço do Petróleo Brent")
plt.show()

# Calcular RMSE corretamente
rmse = mean_squared_error(test["Preço (US$)"], test["Previsão"]) ** 0.5
print(f"RMSE (Prophet): {rmse:.2f}")

# 19. Modelo Prophet + XGBoost para Previsão Ajustada

In [ ]:
try:
    df = pd.read_csv("dados/dados_petroleo_brent_2005_2025.csv")
    print("Arquivo carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    raise

# Verificar colunas esperadas
if "Data" not in df.columns or "Preço (US$)" not in df.columns:
    raise ValueError("O arquivo CSV deve conter as colunas 'Data' e 'Preço (US$)'.")

# Converter colunas para os tipos corretos
df["ds"] = pd.to_datetime(df["Data"], errors="coerce")  # Converte para datetime
df["y"] = pd.to_numeric(df["Preço (US$)"], errors="coerce")  # Converte para numérico

# Verificar valores ausentes
if df["ds"].isnull().any() or df["y"].isnull().any():
    print("Atenção: Há valores ausentes nas colunas 'Data' ou 'Preço (US$)'. Preenchendo valores ausentes...")
    df["ds"].fillna(method="ffill", inplace=True)  # Preenche datas ausentes
    df["y"].fillna(method="ffill", inplace=True)  # Preenche preços ausentes

# Ordenar os dados corretamente
df = df.sort_values(by="ds").reset_index(drop=True)

# Remover duplicatas
if df.duplicated(subset=["ds"]).any():
    print("Atenção: Há duplicatas na coluna 'Data'. Removendo duplicatas...")
    df = df.drop_duplicates(subset=["ds"])

# Exibir as primeiras linhas do DataFrame tratado
df.head()

## Treinamento do Modelo Prophet

In [ ]:
# Criar e treinar o modelo Prophet
prophet = Prophet()
prophet.fit(df[["ds", "y"]])  # Usando apenas as colunas necessárias

# Definir a data final desejada (31 de dezembro de 2026)
data_final_desejada = pd.to_datetime("2026-12-31")

# Calcular o número de dias até essa data
ultima_data_df = df["ds"].max()
dias_ate_2026 = (data_final_desejada - ultima_data_df).days

# Criar previsões do Prophet até o final de 2026
future = prophet.make_future_dataframe(periods=dias_ate_2026)
prophet_future = prophet.predict(future)

# Mesclar previsões do Prophet com o DataFrame original
df = df.merge(prophet_future[["ds", "yhat", "yhat_lower", "yhat_upper"]], on="ds", how="left")

# Exibir as 5 primeiras previsões do Prophet
df.head()

## Ajuste das Previsões com XGBoost

In [ ]:
#Renomear colunas para facilitar a interpretação
df.rename(
    columns={
        "ds": "Data",
        "y": "Preço Real",
        "yhat": "Preço Previsto",
        "yhat_lower": "Intervalo Inferior",
        "yhat_upper": "Intervalo Superior",
    },
    inplace=True,
)


# Criar a coluna de resíduos
df["Resíduo"] = df["Preço Real"] - df["Preço Previsto"]
# Remover colunas duplicadas após a renomeação
df = df.loc[:, ~df.columns.duplicated()]
# Criar features para o modelo XGBoost (lags dos resíduos)
for i in range(1, 8):  # Criar lags de 1 a 7 dias
    df[f"Resíduo_Lag_{i}"] = df["Resíduo"].shift(i)

# Remover linhas com valores ausentes gerados pelos lags
df.dropna(inplace=True)

# Dividir os dados em treino e teste (80% para treino)
train_size = int(len(df) * 0.8)
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# Definir features e target para XGBoost
features = [f"Resíduo_Lag_{i}" for i in range(1, 8)]
X_train, y_train = train[features], train["Resíduo"]
X_test, y_test = test[features], test["Resíduo"]

# Treinar o modelo XGBoost
model_xgb = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, random_state=42)
model_xgb.fit(X_train, y_train)

# Fazer previsões com XGBoost
y_pred_residuo = model_xgb.predict(X_test)

# Ajustar as previsões do Prophet com os resíduos previstos pelo XGBoost
test["Preço Previsto Ajustado"] = test["Preço Previsto"] + y_pred_residuo

# Exibir as primeiras previsões ajustadas
test.head()

## Avaliação do Modelo

In [ ]:
# Calcular métricas de desempenho para as previsões ajustadas
rmse_ajustado = np.sqrt(mean_squared_error(test['Preço Real'], test['Preço Previsto Ajustado']))
mae_ajustado = mean_absolute_error(test['Preço Real'], test['Preço Previsto Ajustado'])
mape_ajustado = np.mean(np.abs((test['Preço Real'] - test['Preço Previsto Ajustado']) / test['Preço Real'])) * 100

print(f"RMSE (Ajustado): {rmse_ajustado}")
print(f"MAE (Ajustado): {mae_ajustado}")
print(f"MAPE (Ajustado): {mape_ajustado}%")

# # Salvar os modelos treinados
# joblib.dump(prophet, 'modelo_prophet.pkl')  # Salvar o modelo Prophet
# joblib.dump(model_xgb, 'modelo_xgboost.pkl')  # Salvar o modelo XGBoost
# print("Modelos salvos com sucesso!")

# Criar previsões para 2025-2026
future_df = prophet_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
future_df = future_df[future_df['ds'] >= '2025-01-01']  # Filtrar apenas anos futuros

# Criar features de calendário para o futuro
future_df['year'] = future_df['ds'].dt.year
future_df['month'] = future_df['ds'].dt.month
future_df['day'] = future_df['ds'].dt.day
future_df['dayofweek'] = future_df['ds'].dt.dayofweek

# Criar lags com os últimos dados disponíveis
for i in range(1, 8):
    future_df[f'Resíduo_Lag_{i}'] = df[f'Resíduo_Lag_{i}'].iloc[-1]

# Aplicar XGBoost para correção da previsão de 2025-2026
future_X = future_df[features]
y_pred_residuo_future = model_xgb.predict(future_X)
future_df['Preço Previsto Ajustado'] = future_df['yhat'] + y_pred_residuo_future

# Criar intervalo de confiança para a previsão corrigida
future_df['Preço Previsto Ajustado Inferior'] = future_df['yhat_lower'] + y_pred_residuo_future
future_df['Preço Previsto Ajustado Superior'] = future_df['yhat_upper'] + y_pred_residuo_future

# Criar título com métricas
title_text = (f"Correção de Previsão Prophet com XGBoost\n"
              f"RMSE: {rmse_ajustado:.2f}, MAE: {mae_ajustado:.2f}, MAPE: {mape_ajustado:.2f}%")

## Visualização das Previsões

In [ ]:

# Plotar resultados
plt.figure(figsize=(14, 6))
df['Data'] = pd.to_datetime(df['Data'])
test['Data'] = pd.to_datetime(test['Data'])
future_df['ds'] = pd.to_datetime(future_df['ds'])

# Adicionando o histórico completo ao gráfico
plt.plot(df['Data'], df['Preço Real'], label="Histórico Completo (Real)", color="blue", linewidth=1.2)

# Adicionando os dados reais do conjunto de teste
plt.plot(test['Data'], test['Preço Real'], label="Teste (Real)", color="green", linewidth=1.5)

# Adicionando a previsão do Prophet
plt.plot(df['Data'], df['Preço Previsto'], label="Previsão Prophet", linestyle="--", color="orange", linewidth=1.5)

# Adicionando a previsão corrigida
plt.plot(test['Data'], test['Preço Previsto Ajustado'], label="Previsão Corrigida (Prophet + XGBoost)", linestyle="-", color="purple", linewidth=1.5)

plt.plot(future_df['ds'], future_df['Preço Previsto Ajustado'], label="Previsão Futura Corrigida (2025-2026)", linestyle="-", color="red", linewidth=1.5)

# Adicionar intervalo de confiança
plt.fill_between(future_df['ds'], future_df['Preço Previsto Ajustado Inferior'], future_df['Preço Previsto Ajustado Superior'], color='gray', alpha=0.3, label="Intervalo de Confiança")

plt.title(title_text)
plt.xlabel("Data")
plt.ylabel("Preço (US$)")
plt.grid()
plt.legend()
plt.show()

## Salvar as Previsões

In [ ]:
# Criar tabela com as previsões futuras (2025-2026)
future_df = prophet_future[["ds", "yhat", "yhat_lower", "yhat_upper"]].copy()
future_df = future_df[future_df["ds"] >= "2025-01-01"]  # Filtrar apenas anos futuros

# Criar lags para previsões futuras
for i in range(1, 8):
    future_df[f"Resíduo_Lag_{i}"] = df[f"Resíduo_Lag_{i}"].iloc[-1]

# Aplicar XGBoost para correção da previsão futura
future_X = future_df[features]
y_pred_residuo_future = model_xgb.predict(future_X)
future_df["Preço Previsto Ajustado"] = future_df["yhat"] + y_pred_residuo_future

# Salvar as previsões ajustadas
future_df.to_csv("dados/previsoes_futuras.csv", index=False)
print("Previsões futuras salvas com sucesso!")

In [ ]:
#Carregar os dados
try:
    df = pd.read_csv("dados/dados_petroleo_brent_2005_2025.csv")
    print("Arquivo carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o arquivo: {e}")
    raise

# Verificar colunas esperadas
if 'Data' not in df.columns or 'Preço (US$)' not in df.columns:
    raise ValueError("O arquivo CSV deve conter as colunas 'Data' e 'Preço (US$)'.")

# Converter colunas para os tipos corretos
df['ds'] = pd.to_datetime(df['Data'], errors='coerce')  # Converte para datetime
df['y'] = pd.to_numeric(df['Preço (US$)'], errors='coerce')  # Converte para numérico

# Verificar valores ausentes
if df['ds'].isnull().any() or df['y'].isnull().any():
    print("Atenção: Há valores ausentes nas colunas 'Data' ou 'Preço (US$)'. Preenchendo valores ausentes...")
    df['ds'].fillna(method='ffill', inplace=True)  # Preenche datas ausentes
    df['y'].fillna(method='ffill', inplace=True)  # Preenche preços ausentes

# Ordenar os dados corretamente
df = df.sort_values(by='ds').reset_index(drop=True)

# Verificar e remover duplicatas na coluna 'ds' (Data)
if df.duplicated(subset=['ds']).any():
    print("Atenção: Há duplicatas na coluna 'Data'. Removendo duplicatas...")
    df = df.drop_duplicates(subset=['ds'])

# Treinar o modelo Prophet
prophet = Prophet()
prophet.fit(df[['ds', 'y']])  # Usando apenas as colunas 'ds' e 'y'

# Definir a data final desejada (31 de dezembro de 2026)
data_final_desejada = pd.to_datetime('2026-12-31')

# Calcular o número de dias até a data final desejada
ultima_data_df = df['ds'].max()
dias_ate_2026 = (data_final_desejada - ultima_data_df).days

# Criar previsões do Prophet até o final de 2026
future = prophet.make_future_dataframe(periods=dias_ate_2026)  
prophet_future = prophet.predict(future)

# Mesclar previsões do Prophet com o DataFrame original
df = df.merge(prophet_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds', how='left')

# Verificar duplicatas após a mesclagem
if df.duplicated(subset=['ds']).any():
    print("Atenção: Há duplicatas após a mesclagem. Removendo duplicatas...")
    df = df.drop_duplicates(subset=['ds'])

# Renomear as colunas de forma clara e estruturada
mapeamento_colunas = {
    'ds': 'Data',
    'y': 'Preço Real',
    'yhat': 'Preço Previsto',
    'yhat_lower': 'Intervalo Inferior',
    'yhat_upper': 'Intervalo Superior'
}

df.rename(columns=mapeamento_colunas, inplace=True)

# Remover colunas duplicadas após a renomeação
df = df.loc[:, ~df.columns.duplicated()]

# Calcular resíduos (erros) do Prophet
df['Resíduo'] = df['Preço Real'] - df['Preço Previsto']

# Criar features para o modelo XGBoost
# Adicionar lags dos resíduos como features
for i in range(1, 8): 
    df[f'Resíduo_Lag_{i}'] = df['Resíduo'].shift(i)

# Remover linhas com valores ausentes gerados pelos lags
df.dropna(inplace=True)

# Dividir os dados em treino e teste
train_size = int(len(df) * 0.8)  # 80% para treino, 20% para teste
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# Definir features e target para o XGBoost
features = [f'Resíduo_Lag_{i}' for i in range(1, 8)]  
X_train = train[features]
y_train = train['Resíduo']
X_test = test[features]
y_test = test['Resíduo']

# Treinar o modelo XGBoost
model_xgb = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)
model_xgb.fit(X_train, y_train)

# Fazer previsões com o XGBoost
y_pred_residuo = model_xgb.predict(X_test)

# Ajustar as previsões do Prophet com os resíduos previstos pelo XGBoost
test['Preço Previsto Ajustado'] = test['Preço Previsto'] + y_pred_residuo

# Calcular métricas de desempenho para as previsões ajustadas
rmse_ajustado = np.sqrt(mean_squared_error(test['Preço Real'], test['Preço Previsto Ajustado']))
mae_ajustado = mean_absolute_error(test['Preço Real'], test['Preço Previsto Ajustado'])
mape_ajustado = np.mean(np.abs((test['Preço Real'] - test['Preço Previsto Ajustado']) / test['Preço Real'])) * 100

print(f"RMSE (Ajustado): {rmse_ajustado}")
print(f"MAE (Ajustado): {mae_ajustado}")
print(f"MAPE (Ajustado): {mape_ajustado}%")

# Salvar os modelos treinados
joblib.dump(prophet, 'dados/modelo_prophet.pkl')  
joblib.dump(model_xgb, 'dados/modelo_xgboost.pkl')  
print("Modelos salvos com sucesso!")

#  Criar previsões para 2025-2026
future_df = prophet_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].copy()
future_df = future_df[future_df['ds'] >= '2025-01-01']  # Filtrar apenas anos futuros

# Criar features de calendário para o futuro
future_df['year'] = future_df['ds'].dt.year
future_df['month'] = future_df['ds'].dt.month
future_df['day'] = future_df['ds'].dt.day
future_df['dayofweek'] = future_df['ds'].dt.dayofweek

# Criar lags com os últimos dados disponíveis
for i in range(1, 8):
    future_df[f'Resíduo_Lag_{i}'] = df[f'Resíduo_Lag_{i}'].iloc[-1]

# Aplicar XGBoost para correção da previsão de 2025-2026
future_X = future_df[features]
y_pred_residuo_future = model_xgb.predict(future_X)
future_df['Preço Previsto Ajustado'] = future_df['yhat'] + y_pred_residuo_future

# Criar intervalo de confiança para a previsão corrigida
future_df['Preço Previsto Ajustado Inferior'] = future_df['yhat_lower'] + y_pred_residuo_future
future_df['Preço Previsto Ajustado Superior'] = future_df['yhat_upper'] + y_pred_residuo_future

# Criar título com métricas
title_text = (f"Correção de Previsão Prophet com XGBoost\n"
              f"RMSE: {rmse_ajustado:.2f}, MAE: {mae_ajustado:.2f}, MAPE: {mape_ajustado:.2f}%")


# Plotar resultados
plt.figure(figsize=(14, 6))
df['Data'] = pd.to_datetime(df['Data'])
test['Data'] = pd.to_datetime(test['Data'])
future_df['ds'] = pd.to_datetime(future_df['ds'])

# Adicionando o histórico completo ao gráfico
plt.plot(df['Data'], df['Preço Real'], label="Histórico Completo (Real)", color="blue", linewidth=1.2)

# Adicionando os dados reais do conjunto de teste
plt.plot(test['Data'], test['Preço Real'], label="Teste (Real)", color="green", linewidth=1.5)

# Adicionando a previsão do Prophet
plt.plot(df['Data'], df['Preço Previsto'], label="Previsão Prophet", linestyle="--", color="orange", linewidth=1.5)

# Adicionando a previsão corrigida
plt.plot(test['Data'], test['Preço Previsto Ajustado'], label="Previsão Corrigida (Prophet + XGBoost)", linestyle="-", color="purple", linewidth=1.5)


plt.plot(future_df['ds'], future_df['Preço Previsto Ajustado'], label="Previsão Futura Corrigida (2025-2026)", linestyle="-", color="red", linewidth=1.5)

# Adicionar intervalo de confiança
plt.fill_between(future_df['ds'], future_df['Preço Previsto Ajustado Inferior'], future_df['Preço Previsto Ajustado Superior'], color='gray', alpha=0.3, label="Intervalo de Confiança")

plt.title(title_text)
plt.xlabel("Data")
plt.ylabel("Preço (US$)")
plt.grid()
plt.legend()
plt.show()


# Criar tabela com as previsões futuras
tabela_previsoes = future_df[['ds', 'Preço Previsto Ajustado', 'Preço Previsto Ajustado Inferior', 'Preço Previsto Ajustado Superior']]
tabela_previsoes.columns = ['Data', 'Preço Previsto Ajustado', 'Intervalo Inferior', 'Intervalo Superior']

# Exibir a tabela no console
print("\nTabela de Previsões Futuras (2025-2026):")
print(tabela_previsoes)

# Salvar a tabela como um arquivo CSV
tabela_previsoes.to_csv('dados/previsoes_futuras.csv', index=False)
print("\nTabela de previsões salva como 'previsoes_futuras.csv'.")

# 20. Criar funcao para gerar tabela automaticamente

In [165]:
# Função para criar tabela de previsões a partir de uma data específica
def criar_tabela_previsoes(data_inicio, dias_futuros, df_inicial):
    """
    Função para criar uma tabela de previsões a partir de uma data específica.
    :param data_inicio: Data inicial no formato 'YYYY-MM-DD'.
    :param dias_futuros: Número de dias para prever no futuro.
    :param df_inicial: DataFrame inicial com os dados históricos.
    :return: DataFrame com as previsões.
    """
    # Carregar os modelos salvos
    prophet = joblib.load('modelo/modelo_prophet.pkl')
    model_xgb = joblib.load('modelo/modelo_xgboost.pkl')

    # Criar DataFrame com as datas futuras
    datas_futuras = pd.date_range(start=data_inicio, periods=dias_futuros, freq='D')
    future_df = pd.DataFrame({'ds': datas_futuras})

    # Fazer previsões com o Prophet
    prophet_future = prophet.predict(future_df)

    # Mesclar previsões do Prophet com o DataFrame futuro
    future_df = future_df.merge(prophet_future[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], on='ds', how='left')

    # Calcular resíduos previstos pelo XGBoost
    # Para isso, precisamos dos últimos 7 resíduos históricos
    ultimos_residuos = df_inicial['Resíduo'].tail(7).values
    if len(ultimos_residuos) < 7:
        raise ValueError("Não há dados históricos suficientes para prever o resíduo.")

    # Criar features para o XGBoost
    features_xgb = {f'Resíduo_Lag_{i+1}': ultimos_residuos[-(i+1)] for i in range(7)}
    features_xgb = pd.DataFrame([features_xgb])

    # Prever o resíduo com o XGBoost
    residuo_previsto = model_xgb.predict(features_xgb)[0]

    # Ajustar as previsões do Prophet com o resíduo previsto
    future_df['Preço Previsto Ajustado'] = future_df['yhat'] + residuo_previsto
    future_df['Preço Previsto Ajustado Inferior'] = future_df['yhat_lower'] + residuo_previsto
    future_df['Preço Previsto Ajustado Superior'] = future_df['yhat_upper'] + residuo_previsto

    # Renomear colunas
    future_df.rename(columns={
        'ds': 'Data',
        'Preço Previsto Ajustado': 'Preço Previsto',
        'Preço Previsto Ajustado Inferior': 'Valor Mínimo Esperado',
        'Preço Previsto Ajustado Superior': 'Valor Máximo Esperado'
    }, inplace=True)

    # Arredondar os valores para 2 casas decimais
    future_df = future_df.round(2)

    # Selecionar colunas relevantes
    tabela_previsoes = future_df[['Data', 'Preço Previsto', 'Valor Mínimo Esperado', 'Valor Máximo Esperado']]

    return tabela_previsoes




### TESTANDO FUNCAO

In [ ]:
#Testar a função de criar tabela de previsões
data_inicio = '2025-01-01'  # Data inicial para previsões
dias_futuros = 30  # Número de dias para prever
try:
    tabela_previsoes = criar_tabela_previsoes(data_inicio, dias_futuros, df)
    print("\nTabela de Previsões Futuras:")
    print(tabela_previsoes)
except Exception as e:
    print(f"Erro ao criar tabela de previsões: {e}")